In [ ]:
import cv2
import numpy as np
import os

# Ruta de la carpeta que contiene las imágenes
folder_path = "C:/Users/Ximo/Desktop/gt"

# Define the BGR value for blue
blue_bgr = np.array([142, 0, 0])

# Define a threshold for black pixels
black_threshold = 30  # Adjust this value based on your image

# Iterar a través de todas las imágenes en la carpeta
for filename in os.listdir(folder_path):
    if filename.endswith(".png"):
        # Load the image
        image_path = os.path.join(folder_path, filename)
        image = cv2.imread(image_path)

        # Create a mask for the specified blue color
        blue_mask = np.all(image == blue_bgr, axis=-1)

        # Find contours in the blue mask
        contours, _ = cv2.findContours(np.uint8(blue_mask), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Iterate through the contours
        for contour in contours:
            # Calculate the area of the contour
            area = cv2.contourArea(contour)

            # Calculate the perimeter of the contour
            perimeter = cv2.arcLength(contour, True)

            # Approximate the contour to a polygon
            approx = cv2.approxPolyDP(contour, 0.02 * perimeter, True)

            # Get the bounding box of the contour
            x, y, w, h = cv2.boundingRect(contour)

            # Extract the region of interest (ROI) for the bounding box
            roi = image[y:y+h, x:x+w]

            # Calculate the number of black pixels in the ROI
            black_pixels = np.sum(roi < black_threshold)

            # Calculate the total number of pixels in the ROI
            total_pixels = roi.size

            # Calculate the percentage of black pixels in the ROI
            roi_percentage = black_pixels / total_pixels

            # If 70% or more of the pixels in the ROI are black, fill the area with yellow
            if roi_percentage >= 0.7:
                cv2.drawContours(image, [contour], -1, (0, 255, 255), -1)

        # Save the result with a new filename
        result_filename = f"{os.path.splitext(filename)[0]}_aparcado.png"
        result_path = os.path.join(folder_path, result_filename)
        cv2.imwrite(result_path, image)
